Download spreadsheet from gdrive. [Tutorial](https://developers.google.com/sheets/api/quickstart/python)

# Download spread sheets

In [8]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


  Obtaining dependency information for google-auth from https://files.pythonhosted.org/packages/9d/47/603554949a37bca5b7f894d51896a9c534b9eab808e2520a748e081669d0/google_auth-2.38.0-py2.py3-none-any.whl.metadata
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Obtaining dependency information for google-auth-oauthlib from https://files.pythonhosted.org/packages/1a/8e/22a28dfbd218033e4eeaf3a0533b2b54852b6530da0c0fe934f0cc494b29/google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2 from https://files.pythonhosted.org/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/9d/dc/432b2b61e2335dc20196f543b5c84400d246c8cdfb8169fea183e7a0c52d/


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from os import path

from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1xXZAZ4lm1Jg4458QOUxxYt7ZBYtegm4ECiYEkvTJBIE"
SAMPLE_RANGE_NAME = "Class Data!A2:E"
TOKEN_FILE = r"..\google_token.json"

In [ ]:
creds = None

# if path.exists(TOKEN_FILE):
#     creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(TOKEN_FILE, SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open(TOKEN_FILE, "w") as token:
        token.write(creds.to_json())

try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID).execute()
    values = result.get("values", [])

    if not values:
        print("No data found.")
        return

    print("Name, Major:")
    for row in values:
        # Print columns A and E, which correspond to indices 0 and 4.
        print("%s, %s" % (row[0], row[4]))
except HttpError as err:
    print(err)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=369946212657-bfoh9k0flmmg10drrb0lq1h39v4kgia5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=MMqoWXu15dEhZg1XPJWFxnQpoSA0Nh&access_type=offline


# Parse

In [2]:
import pandas as pd
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

In [8]:
def process_xlsx(path: str, meta: str):
    excel = pd.ExcelFile(path)
    words = []
    for sheet_name in excel.sheet_names:
        sheet_df = pd.read_excel(excel, sheet_name=sheet_name, skiprows=1, header=None)
        print(sheet_name, sheet_df.shape)
        for col in sheet_df.columns[:2]:  # sometimes experts can put number in random column
            # 0 - base, 1 - advanced
            for word in sheet_df.loc[sheet_df[col].notna(), col]:
                word = str(word)
                weight = 1
                try:
                    if word[0] == "!":
                        weight = 5
                        word = word[1:]
                except Exception as e:
                    print(sheet_name, word, e)
                words.append(
                    {
                        "meta": meta,
                        "topic": sheet_name,
                        "weight": weight,
                        "type": col,
                        "word": morph.normal_forms(word)[0],
                    }
                )
    return words

In [9]:
df = []
print("E")
df.extend(process_xlsx("../topics/Environmental.xlsx", "env"))
print("S")
df.extend(process_xlsx("../topics/Social.xlsx", "social"))
print("G")
df.extend(process_xlsx("../topics/Governance.xlsx", "gov"))
df = pd.DataFrame(df)
df.head()

E
Экология в целом (21, 2)
Климат (152, 2)
+Энергия (169, 2)
+Воздух (96, 2)
+Вода (108, 2)
+Отходы и циклическая экономика (98, 2)
+Биоразнообразие (112, 2)
Рекультивация земель (75, 2)
Экологичность продукта (61, 2)
S
!Персонал в целом (14, 6)
Обучение и развитие (139, 6)
Сотрудники. Вовлеченность и мот (28, 2)
Оплата труда (92, 5)
Сотрудники. Здоровье и благопол (54, 3)
Сотрудники. Наем и увольнение (45, 5)
Сотрудники. Корпоративная культ (38, 4)
Сотрудники. Безопасность и охра (131, 4)
Сотрудники. Профсоюз и Коллекти (66, 4)
Потребители. Доступность (56, 1)
Потребители. Сервис и коммуника (80, 3)
Потребители. Персональные данны (62, 2)
Потребители. Здоровье и благопо (38, 2)
Потребители. Маркетинг, продажи (100, 2)
Потребители. Удовлетворенность (72, 4)
Потребители. Качество и безопас (115, 7)
Потребители. Ценовая политика (81, 4)
Поставщики в целом (30, 1)
Малый и локальный бизнес (32, 4)
Поставщики. Работники (23, 4)
Поставщики. Экология (34, 2)
Закупки и антикоррупция (84, 4)
За

,meta,topic,weight,type,word
0,env,Экология в целом,5,0,воздействие
1,env,Экология в целом,5,0,окружающая среда
2,env,Экология в целом,5,0,природа
3,env,Экология в целом,5,0,природоохранный
4,env,Экология в целом,5,0,экологический


In [10]:
df = df[~df["word"].str.strip().eq("")]

In [12]:
df.to_csv("topic_words.csv.zip", index=False)
df.to_csv("topic_words.csv", index=False)

In [1]:
import numpy as np
data = np.load('expert.npy')
data

array([[ -110., -1160.,  -650., ...,   -20., -1260., -3160.],
       [ -110., -1160.,  -650., ...,   -19., -1259., -3159.],
       [ -110., -1160.,  -650., ...,   -20., -1260., -3160.],
       ...,
       [   31.,    61.,  -109., ...,   -20., -1240., -2580.],
       [ -110., -1120.,  -630., ...,   -20., -1220., -3100.],
       [ -110., -1160.,  -650., ...,   -20., -1220., -3040.]],
      shape=(45, 3775))